<a href="https://colab.research.google.com/github/ferozferoz/DeepLearn/blob/master/Emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# this code learns sentiments and return emojis based on the sentence sentiment score
# this code is re production of the Emoji example demonstrated by Andrew Ng course in coursera
import numpy as np
import requests
from contextlib import closing
import csv
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def read_csv(filename):
    phrase = []
    emoji = []

    with closing(requests.get(filename, stream=True)) as r:
      f = (line.decode('utf-8') for line in r.iter_lines())
      reader = csv.reader(f, delimiter=',', quotechar='"')
      for row in reader:
        
        phrase.append(row[0])
        emoji.append(row[1])
    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)
    return X, Y

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


In [34]:
X_train, Y_train = read_csv('https://raw.githubusercontent.com/AdalbertoCq/Deep-Learning-Specialization-Coursera/master/Sequence%20Models/week2/Emojyfier/data/emojify_data.csv')
X_test, Y_test = read_csv('https://raw.githubusercontent.com/AdalbertoCq/Deep-Learning-Specialization-Coursera/master/Sequence%20Models/week2/Emojyfier/data/test_emoji.csv')

print(X_train.shape)
print(Y_train.shape)


print(X_train[0])
print(Y_train[0])



(183,)
(183,)
French macaroon is so tasty
4


In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/gdrive/My Drive/ColabNotebooks/glove.6B.50d.txt')

In [14]:
# printing all three variables for observation
print(word_to_index['a'])
print("+++++++++++++++++++++++")
print(index_to_word[1])
print("+++++++++++++++++++++++")
print(word_to_vec_map['a'])

43010
+++++++++++++++++++++++
!
+++++++++++++++++++++++
[ 0.21705   0.46515  -0.46757   0.10082   1.0135    0.74845  -0.53104
 -0.26256   0.16812   0.13182  -0.24909  -0.44185  -0.21739   0.51004
  0.13448  -0.43141  -0.03123   0.20674  -0.78138  -0.20148  -0.097401
  0.16088  -0.61836  -0.18504  -0.12461  -2.2526   -0.22321   0.5043
  0.32257   0.15313   3.9636   -0.71365  -0.67012   0.28388   0.21738
  0.14433   0.25926   0.23434   0.4274   -0.44451   0.13813   0.36973
 -0.64289   0.024142 -0.039315 -0.26037   0.12017  -0.043782  0.41013
  0.1796  ]


In [36]:
# function will convert a corpus of sentence into corpus of indices padded by max_len argument
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):
      sentence_words = [w.lower() for w in X[i].split()]
      j = 0
      for w in sentence_words:
        X_indices[i, j] = word_to_index[w]
        j += 1
    return X_indices
x_indices = sentences_to_indices(X_train,word_to_index,10)
x_indices[0]

array([153730., 229211., 192973., 336114., 353730.,      0.,      0.,
            0.,      0.,      0.])

In [0]:
# this function will return a embedding layer that will be used in later functions
def pre_embedded_layer(word_to_vec_map,word_to_index):
  #Embedding row is always 1 greater than vocabulary size
  embedding_rows = len(word_to_index)+1
  embedding_columns = word_to_vec_map['a'].shape[0]
  e_matrix = np.zeros((embedding_rows, embedding_columns))
  for word,index in word_to_index.items():
    e_matrix[index,:] = word_to_vec_map[word]
  
  embedding_layer = Embedding(embedding_rows,embedding_columns,trainable=False)
  embedding_layer.build((None,))
  embedding_layer.set_weights([e_matrix])
  return embedding_layer


In [0]:

def emoji_model(input_shape, word_to_vec_map, word_to_index):
    
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pre_embedded_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [39]:

max_len = 10
model = emoji_model((max_len,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645 

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [42]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_len)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)


Epoch 1/50
183/183 [==============================] - 0s 816us/step - loss: 0.5544 - acc: 0.7923
Epoch 2/50
183/183 [==============================] - 0s 798us/step - loss: 0.4888 - acc: 0.8033
Epoch 3/50
183/183 [==============================] - 0s 848us/step - loss: 0.4285 - acc: 0.8415
Epoch 4/50
183/183 [==============================] - 0s 847us/step - loss: 0.4040 - acc: 0.8579
Epoch 5/50
183/183 [==============================] - 0s 814us/step - loss: 0.3590 - acc: 0.8689
Epoch 6/50
183/183 [==============================] - 0s 846us/step - loss: 0.3654 - acc: 0.8743
Epoch 7/50
183/183 [==============================] - 0s 826us/step - loss: 0.3677 - acc: 0.8579
Epoch 8/50
183/183 [==============================] - 0s 874us/step - loss: 0.2486 - acc: 0.9016
Epoch 9/50
183/183 [==============================] - 0s 767us/step - loss: 0.2398 - acc: 0.9016
Epoch 10/50
183/183 [==============================] - 0s 895us/step - loss: 0.2035 - acc: 0.9290
Epoch 11/50
183/183 [========

In [43]:
loss, acc = model.evaluate(X_train_indices, Y_train_oh)
print()
print("Train accuracy = ", acc)

183/183 [==============================] - 0s 2ms/step

Train accuracy =  1.0


In [44]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 280us/step

Test accuracy =  0.9821428571428571
